run_qNN_notebook.ipynb

Notebook that can run the qNN with two qubits

Dependencies:
- Uses numpy to get random values
- Uses qNN_functions to get qNN necessary functions 

Since:
- 02/2025

Authors:
- Pedro C. Delbem. <pedrodelbem@usp.br>

#imports

In [10]:
import numpy as np #type: ignore
import qNN_functions as tqNN

#generate all inputs circuit

In [11]:
all_inputs_circuit = tqNN.all_inputs_circuit()
all_inputs_circuit.draw()

┌────────────────────────┐
q_0: ┤0                       ├
     │  TwoLocal(π,π,π/2,π/2) │
q_1: ┤1                       ├
     └────────────────────────┘

generate a circuit with generate all possible inputs with 2 bits

function:

def all_inputs_circuit():
    """
    Generates a quantum circuit that produces all the possible inputs.

    The circuit is constructed by using the TwoLocal method. The parameters are set to the previously chosen/learned parameters to generate all the possible inputs.

    Returns:
    quantum_circuit (QuantumCircuit): The quantum circuit that generates all the possible inputs.
    """

    two_qubits_entanglement_circuit = TwoLocal(2, "rx", "cz", entanglement="linear", reps=1) #generates a circuit that mixture two qubits - an entanglement circuit
    parameters = [pi, pi, pi/2, pi/2] #previously chosen/learned parameters to generate all the possible inputs
    parameter_dict = dict(zip(two_qubits_entanglement_circuit.parameters, parameters)) #assigns the parameters to the circuit
    initial_states_circuit = two_qubits_entanglement_circuit.assign_parameters(parameter_dict) #a circuit that can generate all the possible inputs

    #return the circuit
    return initial_states_circuit

#copy the circuit

In [12]:
duplicate_circuit = tqNN.circuit_copy(all_inputs_circuit,2)
duplicate_circuit.draw()

┌────────────────────────┐ ░                     
q_0: ┤0                       ├─░────────■────────────
     │  TwoLocal(π,π,π/2,π/2) │ ░        │            
q_1: ┤1                       ├─░────────┼────■───────
     └────────────────────────┘ ░ ┌───┐┌─┴─┐  │  ┌───┐
q_2: ───────────────────────────░─┤ X ├┤ X ├──┼──┤ X ├
                                ░ ├───┤└───┘┌─┴─┐├───┤
q_3: ───────────────────────────░─┤ X ├─────┤ X ├┤ X ├
                                ░ └───┘     └───┘└───┘

duplicate the circuit to be able to process the inputs without losing the information about which input was used

function:
def circuit_copy(initial_circuit, number_of_qubits):
    """
    Creates a quantum circuit that duplicates the given initial circuit with additional qubits.

    This function constructs a new quantum circuit by duplicating the specified number of qubits 
    from the initial circuit and adding two additional qubits to the circuit. The resulting circuit 
    reproduces the input values using controlled-NOT operations to copy the state of the first two 
    qubits to the additional qubits.

    Parameters:
    initial_circuit (QuantumCircuit): The initial quantum circuit to be copied.
    number_of_qubits (int): The number of qubits in the initial circuit.

    Returns:
    QuantumCircuit: A new quantum circuit with duplicated qubits and additional operations to copy the input.
    """

    circuit_copy = QuantumCircuit(number_of_qubits+2) #duplicate the number of qubits
    circuit_copy = circuit_copy .compose(initial_circuit, qubits=list(range(0,number_of_qubits))) #first half with the initial_states_2b_circ
    circuit_copy.barrier() #to visually separate circuit components

    #a layer to copy/reproduce the generated inputs in qubits 0 and 1
    circuit_copy.x(2)    #change value of qubit 2 from 0 to 1
    circuit_copy.x(3)    #change value of qubit 3 from 0 to 1
    circuit_copy.cx(0,2) #qb0 ''AND'' 1 (or NOT qb0) to copy qubit 0 to qubit 2
    circuit_copy.cx(1,3) #qb1 ''AND'' 1 (or NOT qb1) to copy qubit 1 to qubit 3
    circuit_copy.x(2)    #NOT of qubit 2 => qubit 2 equal to equal qubit 0
    circuit_copy.x(3)    #NOT of qubit 3 => qubit 3 equal to equal qubit 1

    return circuit_copy 

#generate the qNN circuit

In [13]:
#generate 4 random angles
angles = np.random.rand(4)*np.pi
angles

array([2.90185749, 2.03046355, 2.05952533, 2.40280398])

In [14]:
qNN_circuit = tqNN.qNN_circuit(all_inputs_circuit,angles)
qNN_circuit.print_circuit()

        ┌───────────────┐                                                 ░ ┌─┐»
   q_0: ┤0              ├─────────────────────────────────────────────────░─┤M├»
        │               │                                                 ░ └╥┘»
   q_1: ┤1              ├─────────────────────────────────────────────────░──╫─»
        │  circuit-1177 │┌────────────┐     ┌────────────┐     ┌───┐      ░  ║ »
   q_2: ┤2              ├┤ Ry(2.9019) ├──■──┤ Ry(2.0595) ├──■──┤ X ├──■───░──╫─»
        │               │├────────────┤┌─┴─┐├────────────┤┌─┴─┐├───┤  │   ░  ║ »
   q_3: ┤3              ├┤ Ry(2.0305) ├┤ X ├┤ Ry(2.4028) ├┤ X ├┤ X ├──■───░──╫─»
        └───────────────┘└────────────┘└───┘└────────────┘└───┘└───┘┌─┴─┐ ░  ║ »
   q_4: ────────────────────────────────────────────────────────────┤ X ├─░──╫─»
                                                                    └───┘ ░  ║ »
   c: 1/═════════════════════════════════════════════════════════════════════╬═»
                            

create the quantum circuit with the four angles with each a Ry rotation mixes both qubit with CNot and multiply both saving in other with CCNot
this structure is applied in the last 2 qubits from the structure with all inputs duplicate

functions:
def four_angle_neuron(qc,weight1,weight2,weight3,weight4,first_qbit_index,first_classical_bit_index):
    """
    Quantum circuit for a neuron.
    
    Parameters:
    qc (QuantumCircuit): The quantum circuit to be modified.
    first_qbit_index (int): The first qubit of the three qubits to be used in the neuron.
    first_classical_bit_index (int): The first classical bit of the three classical bits to be used in the neuron.
    weight1,weight2,weight3,weight4 (float): The weights of the inputs to the neuron.
    """
    
    qc.ry(weight1,first_qbit_index)
    qc.ry(weight2,first_qbit_index+1)
    qc.cx(first_qbit_index,first_qbit_index+1)

    qc.ry(weight3,first_qbit_index)
    qc.ry(weight4,first_qbit_index+1)
    qc.cx(first_qbit_index,first_qbit_index+1)

    qc.x(first_qbit_index)
    qc.x(first_qbit_index+1)
    qc.ccx(first_qbit_index,first_qbit_index+1,first_qbit_index+2)
    
def add_four_angle_neuron(self,weight1,weight2,weight3,weight4,first_qbit_index,first_classical_bit_index): 
    """
    Add a neuron to the current quantum circuit.
        
    Parameters:
    weight1 (float): The weight of the first input to the neuron.
    weight2 (float): The weight of the second input to the neuron.
    weight3 (float): The weight of the third input to the neuron.
    weight4 (float): The weight of the fourth input to the neuron.
    first_qbit_index (int): The index of the first qbit that the neuron will use.
    first_classical_bit_index (int): The index of the first classical bit that the neuron will use.
    """

    four_angle_neuron(self._qc,weight1,weight2,weight3,weight4,first_qbit_index,first_classical_bit_index)

def qNN_circuit(all_inputs_circuit, parameters_of_entanglement_circuit):
    """
    Generates a quantum circuit that produces all the possible inputs and a quantum neural network with two neurons.

    The circuit is constructed by using the TwoLocal method. The parameters are set to the previously chosen/learned parameters to generate all the possible inputs.
    The TwoLocal method generates a circuit that mixture two qubits - an entanglement circuit.
    A quantum neural network with two neurons is added to the circuit, by using the add_bin_neuron3 method of the current_circuit class.
    The parameters of the quantum neural network are set to the previously chosen/learned parameters.

    Parameters:
    all_inputs_circuit (QuantumCircuit): The quantum circuit that generates all the possible inputs.
    parameters_of_entanglement_circuit (list): A list of parameters for the U and controlled-phase (cp) gates.

    Returns:
    quantum_circuit (QuantumCircuit): The quantum circuit with all the possible inputs and a quantum neural network with two neurons.
    """

    qNN = current_circuit(5,1) #create the qNN circuit
    auxiliary_circuit = all_inputs_circuit.copy() #copy the all inputs circuit
    duplicate_circuit = circuit_copy(auxiliary_circuit, 2) #duplicate the all inputs circuit
    qNN.get_current_circuit().append(duplicate_circuit, [0, 1, 2, 3]) #add the all inputs circuit
    qNN.add_four_angle_neuron(*parameters_of_entanglement_circuit, 2, 0) #add the neuron
    qNN.get_current_circuit().measure_all()

    return qNN

#evaluate qNN circuit

In [15]:
counts = qNN_circuit.evaluate(number_of_shots=1024, number_of_runs=1)
counts

[{'11110': 190,
  '00010': 44,
  '00100': 148,
  '01001': 104,
  '00111': 28,
  '00011': 157,
  '01011': 69,
  '11101': 32,
  '00001': 70,
  '00101': 53,
  '01000': 85,
  '11111': 26,
  '00110': 8,
  '00000': 7,
  '11100': 3}]

uses the evaluate structure, from the current circuit class, to obtain results from the circuit
function:

def evaluate(self, number_of_shots = 1024, number_of_runs = 100):
    """
    Evaluate a quantum circuit (XOR candidate) and return the counts (histogram of the outputs).

    Parameters:
    quantum_circuit (QuantumCircuit): The quantum circuit to be evaluated.
    number_of_shots (int): The number of shots to be used in the evaluation.
    number_of_runs (int): The number of times the quantum circuit is run.

    Returns:
    list: A list of dictionaries, where each dictionary represents the counts of the outputs of the quantum circuit.
    """

    #sample results with severals runs, each with several shots
    sampler = StatevectorSampler()
    #create jobs list
    jobs = []
    
    #run the circuit several times
    for _ in range(number_of_runs):

        #run the circuit
        job = sampler.run([(self._qc)], shots = number_of_shots)
        #append the job to the jobs list
        jobs.append(job)

    #create the counts list
    counts = []

    #get and show raw results - counts
    for job in jobs:

        #get the data
        data_pub = job.result()[0].data # 'pub' refers to Primitive Unified Bloc
        job_counts = data_pub.meas.get_counts()

        #append the counts to the counts list
        counts.append(job_counts)

    #return the counts list
    return counts

#compute the error

In [16]:
error = tqNN.error(counts)
error

0.4130859375

computes the circuit error when simulating the XOR gate for given angles

def error(counts):
    """
    Compute the error of the given quantum circuit.

    The error is computed by counting the number of mistakes in the outputs of the quantum circuit.
    The output of the quantum circuit is in the form of a string of length 5, where the first two
    characters are the inputs and the last character is the output. The error is the sum of the
    number of mistakes in the outputs of the quantum circuit divided by the total number of tests.

    Parameters:
    counts (list): A list of dictionaries, where each dictionary represents the counts of the outputs of the quantum circuit.

    Returns:
    float: The error of the quantum circuit.
    """

    #define the statistics dictionary
    statistics = {"00": [0,0], "01": [0,0], "10": [0,0], "11": [0,0]} #defines the statistics dictionary
    
    #get the total number of tests
    total_tests = 0

    for count in counts: #for each count
        for key,value in count.items(): #extract the key and value
            inputs = str(key[2])+str(key[3])
            output = int(key[4])
            statistics[inputs][output] = statistics[inputs][output] + value
            total_tests = total_tests + value

    #compute the error
    error = statistics["00"][1] + statistics["01"][0] + statistics["10"][0] + statistics["11"][1]
    error = error / total_tests

    #return the error
    return error

#do exaustive search

In [17]:
final_parameters, final_error = tqNN.exaustive_grid_search()
final_parameters, final_error

([np.float64(1.0471975511965976),
  np.float64(1.0471975511965976),
  np.float64(2.0943951023931953),
  np.float64(1.0471975511965976)],
 0.1513671875)

do a exaustive search by varying the angles in order to obtain the angle with the lowest error for the XOR gate - obtaning the angles and the error

def exaustive_grid_search(grid_grain=4,number_of_runs=1):
    """
    Perform an exaustive search of the parameter space to find the optimal parameters for the quantum neural network.

    Parameters:
    grid_grain (int): The number of points in the grid to search.
    number_of_runs (int): The number of times the circuit is run for each point in the grid.

    Returns:
    The optimal parameters (list of floats) and the total error (float) of the optimal parameters.
    """
    final_parameters = []
    final_error = 1
    
    for i in np.linspace(0, np.pi, grid_grain):
        for j in np.linspace(0, np.pi, grid_grain):
            for k in np.linspace(0, np.pi, grid_grain):
                for l in np.linspace(0, np.pi, grid_grain):
                    
                    counts = qNN_circuit(all_inputs_circuit(), [i, j, k, l]).evaluate(number_of_runs=number_of_runs)
                    current_error = error(counts)

                    if current_error < final_error:
                        final_error = current_error
                        final_parameters = [i, j, k, l]
                    
                    print(i, j, k, l, current_error)

    return final_parameters, final_error